# Dynamic VBD Testing and Rankings Generation

This notebook demonstrates how to test the **Dynamic VBD (Value-Based Drafting)** integration and generate player rankings tables under different draft scenarios.

## What is Dynamic VBD?

**Traditional (Static) VBD** uses fixed replacement-level baselines that don't change during your draft:
- **VOLS**: Teams × Starters (e.g., 14 teams × 2 RBs = 28th RB is replacement level)
- **VORP**: Teams × (Starters + 1) (e.g., 14 × 3 = 42nd RB is replacement level)
- **BEER**: Teams × (Starters + 0.5) (e.g., 14 × 2.5 = 35th RB is replacement level)

**Dynamic VBD** adjusts these baselines in real-time based on draft flow predictions:
- If you expect a heavy run on RBs, the system **increases** RB baseline values (making RBs more valuable)
- If QBs are being ignored, the system **decreases** QB baseline values (making QBs less valuable relative to other positions)
- This helps you identify the best picks based on what other teams are likely to do next

## Why This Matters for Your Draft

Dynamic VBD helps answer questions like:
- "Should I take this RB now or wait if I think other positions will be drafted heavily?"
- "Is this QB worth taking early if I expect a QB run later?"
- "How does positional scarcity change player values in real-time?"

This notebook tests three realistic draft scenarios to show how Dynamic VBD adapts to different drafting conditions.

In [11]:
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime
import logging

# Setup paths - add both current and parent src directories
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
src_path = os.path.join(parent_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
if '../src' not in sys.path:
    sys.path.append('../src')

print(f"Current directory: {current_dir}")
print(f"Python path includes: {[p for p in sys.path if 'src' in p or 'fantasy' in p]}")

# Import modules with error handling
try:
    from scoring import load_league_config
    print("✅ Successfully imported scoring module")
except ImportError as e:
    print(f"❌ Failed to import scoring: {e}")

try:
    from vbd import calculate_all_vbd_methods
    print("✅ Successfully imported vbd module")
except ImportError as e:
    print(f"❌ Failed to import vbd: {e}")

try:
    from dynamic_vbd import DynamicVBDTransformer, create_probability_forecast, create_draft_state
    print("✅ Successfully imported dynamic_vbd module")
except ImportError as e:
    print(f"❌ Failed to import dynamic_vbd: {e}")

print("✅ Dynamic VBD Testing Setup Complete")

Current directory: /Users/ben/projects/fantasy-football-draft-spreadsheet/notebooks
Python path includes: ['/Users/ben/projects/fantasy-football-draft-spreadsheet/.venv/lib/python3.13/site-packages', '/Users/ben/projects/fantasy-football-draft-spreadsheet/src', '../src']
✅ Successfully imported scoring module
✅ Successfully imported vbd module
✅ Successfully imported dynamic_vbd module
✅ Dynamic VBD Testing Setup Complete


## 1. Load Configuration and Data

In [12]:
# Load league configuration
try:
    config = load_league_config('../config/league-config.yaml')
    if not config:
        raise ValueError("Config is empty")
    print(f"League: {config['basic_settings']['teams']} teams")
    print(f"Dynamic VBD enabled: {config.get('dynamic_vbd', {}).get('enabled', False)}")
    print(f"Dynamic VBD scale: {config.get('dynamic_vbd', {}).get('params', {}).get('scale', 'not set')}")
except Exception as e:
    print(f"❌ Config loading failed: {e}")
    print("📝 Creating minimal config for testing...")
    config = {
        'basic_settings': {'teams': 14},
        'roster': {'roster_slots': {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'DEF': 1, 'K': 1}},
        'dynamic_vbd': {'enabled': True, 'params': {'scale': 25.0, 'kappa': 5.0}},
        'scoring': {'passing': {'yards': 0.04, 'touchdown': 4}, 'rushing': {'yards': 0.1, 'touchdown': 6}}
    }
    print(f"✅ Using minimal config: {config['basic_settings']['teams']} teams")

# Load the latest player projections
try:
    # Try loading from VBD rankings first
    vbd_files = [f for f in os.listdir('../data/output/') if 'vbd_rankings_top300' in f and f.endswith('.csv')]
    if vbd_files:
        latest_vbd = sorted(vbd_files)[-1]
        df = pd.read_csv(f'../data/output/{latest_vbd}')
        print(f"✅ Loaded VBD rankings: {latest_vbd}")
    else:
        # Fallback to raw projections
        proj_files = [f for f in os.listdir('../data/raw/') if 'projections_all_positions' in f and f.endswith('.csv')]
        latest_proj = sorted(proj_files)[-1]
        df = pd.read_csv(f'../data/raw/{latest_proj}')
        print(f"✅ Loaded raw projections: {latest_proj}")
        
        # Calculate fantasy points if not present
        if 'FANTASY_PTS' not in df.columns:
            from scoring import calculate_fantasy_points_vectorized
            df = calculate_fantasy_points_vectorized(df, config)
            print("✅ Calculated fantasy points")
        
        # Calculate static VBD if not present
        if not any('VBD_' in col for col in df.columns):
            df = calculate_all_vbd_methods(df, config)
            print("✅ Calculated static VBD methods")
    
    print(f"Dataset shape: {df.shape}")
    print(f"Available columns: {list(df.columns)}")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("📊 Creating sample data for testing...")
    # Create comprehensive sample data for testing
    positions = ['RB', 'WR', 'QB', 'TE', 'DST', 'K']
    players_per_pos = 50
    
    sample_data = []
    for pos in positions:
        for i in range(players_per_pos):
            base_pts = 200 if pos == 'QB' else 150 if pos in ['RB', 'WR'] else 100
            sample_data.append({
                'PLAYER': f'{pos} Player {i+1}',
                'POSITION': pos,
                'FANTASY_PTS': base_pts - (i * 2) + np.random.normal(0, 5),
                'VBD_BEER': max(0, (base_pts - i * 2) - (base_pts * 0.7)),
                'VBD_VORP': max(0, (base_pts - i * 2) - (base_pts * 0.6)),
                'VBD_VOLS': max(0, (base_pts - i * 2) - (base_pts * 0.8)),
                'VBD_BLENDED': max(0, (base_pts - i * 2) - (base_pts * 0.7))
            })
    
    df = pd.DataFrame(sample_data)
    print(f"✅ Created sample data: {df.shape}")

League: 14 teams
Dynamic VBD enabled: True
Dynamic VBD scale: 25.0
✅ Loaded VBD rankings: vbd_rankings_top300_20250814.csv
Dataset shape: (300, 35)
Available columns: ['PASSING_ATT', 'PASSING_CMP', 'PASSING_YDS', 'PASSING_TDS', 'PASSING_INTS', 'RUSHING_ATT', 'RUSHING_YDS', 'RUSHING_TDS', 'MISC_FL', 'MISC_FPTS', 'POSITION', 'SCRAPE_DATE', 'RECEIVING_REC', 'RECEIVING_YDS', 'RECEIVING_TDS', 'FG', 'FGA', 'XPT', 'FPTS', 'SACK', 'INT', 'FR', 'FF', 'TD', 'SAFETY', 'PA', 'YDS_AGN', 'PLAYER', 'FANTASY_PTS', 'OVERALL_RANK', 'VBD_VOLS', 'VBD_VORP', 'VBD_BEER', 'VBD_BLENDED', 'VBD_RANK']


## 2. Initialize Dynamic VBD System

In [13]:
# Initialize Dynamic VBD transformer
transformer = DynamicVBDTransformer(config)

print(f"Dynamic VBD enabled: {transformer.enabled}")
print(f"Scale parameter: {transformer.scale}")
print(f"Kappa parameter: {transformer.kappa}")

# Show current cache status
cache_stats = transformer.get_cache_stats()
print(f"Cache size: {cache_stats['cache_size']}")

Dynamic VBD enabled: True
Scale parameter: 25.0
Kappa parameter: 5.0
Cache size: 0


## 3. Generate Static VBD Rankings (Baseline)

In [14]:
# Generate static VBD rankings for comparison
df_static = df.copy()

# If VBD columns don't exist, calculate them
if not any('VBD_' in col for col in df_static.columns):
    df_static = calculate_all_vbd_methods(df_static, config)

# Sort by blended VBD or BEER if blended doesn't exist
vbd_col = 'VBD_BLENDED' if 'VBD_BLENDED' in df_static.columns else 'VBD_BEER'
df_static_ranked = df_static.sort_values(vbd_col, ascending=False).reset_index(drop=True)
df_static_ranked['STATIC_RANK'] = range(1, len(df_static_ranked) + 1)

print("📊 STATIC VBD RANKINGS (Top 20)")
print("=" * 60)
static_display = df_static_ranked[['STATIC_RANK', 'PLAYER', 'POSITION', vbd_col]].head(20)
print(static_display.to_string(index=False))

📊 STATIC VBD RANKINGS (Top 20)
 STATIC_RANK                 PLAYER POSITION  VBD_BLENDED
           1     Saquon Barkley PHI       RB       154.80
           2     Bijan Robinson ATL       RB       144.62
           3       Jahmyr Gibbs DET       RB       140.86
           4      Derrick Henry BAL       RB       130.19
           5      Lamar Jackson BAL       QB       113.13
           6         Josh Allen BUF       QB       111.79
           7 Christian McCaffrey SF       RB       110.30
           8         Josh Jacobs GB       RB       108.21
           9      Ja'Marr Chase CIN       WR       106.10
          10      De'Von Achane MIA       RB       105.09
          11       Ashton Jeanty LV       RB        99.26
          12    Jonathan Taylor IND       RB        97.99
          13        Jalen Hurts PHI       QB        95.18
          14     Kyren Williams LAR       RB        95.05
          15     Jayden Daniels WAS       QB        91.06
          16        Bucky Irving TB      

## 4. Test Dynamic VBD Under Different Draft Scenarios

**Understanding the Test Scenarios:**

This notebook tests Dynamic VBD under three different draft scenarios to demonstrate how the system adapts to changing draft conditions. Each scenario represents a different set of expectations about what positions will be drafted in the upcoming picks.

**Why This Matters:**
- Static VBD uses fixed baselines that don't change during the draft
- Dynamic VBD adjusts these baselines in real-time based on draft flow predictions
- Different draft scenarios should produce different player valuations
- This helps identify the best picks based on what you expect other teams to do

**The Three Scenarios:**
- **Scenario A (Balanced)**: Normal early-draft expectations with balanced position distribution
- **Scenario B (RB Run)**: Mid-draft expecting a heavy run on running backs
- **Scenario C (WR/TE Focus)**: Late-draft expecting focus on receivers and tight ends

### Scenario A: Early Draft (Pick 15) - Balanced Expectations

**Context:** You're at pick 15 in the first round of a 14-team draft. This represents typical early-draft behavior where teams are still taking the best available players without major positional runs.

**Draft Probability Assumptions:**
- **RB: 35%** - Running backs are popular early but not dominant
- **WR: 30%** - Wide receivers equally valued in early rounds  
- **QB: 15%** - Some teams reach for QBs early, but not many
- **TE: 15%** - Occasional early TE picks for premium players
- **DST/K: 5%** - Very rare in early rounds

**Expected Impact:** Dynamic VBD should boost QB values significantly since they're being drafted less than their static baseline suggests, while slightly adjusting skill positions based on the balanced expectations.

In [15]:
# Scenario A: Early draft with balanced position probabilities
probabilities_balanced = create_probability_forecast(
    horizon_picks=28,  # Looking ahead 2 rounds (14 teams * 2)
    position_probs={
        'RB': 0.35,
        'WR': 0.30, 
        'QB': 0.15,
        'TE': 0.15,
        'DST': 0.03,
        'K': 0.02
    }
)

draft_state_early = create_draft_state(
    current_pick=15,
    drafted_players=['Saquon Barkley PHI', 'Bijan Robinson ATL']  # Example drafted players
)

print("🔍 DEBUGGING DYNAMIC VBD SCENARIO A")
print("=" * 50)
print(f"Probabilities: {probabilities_balanced.position_probs}")
print(f"Horizon picks: {probabilities_balanced.horizon_picks}")
print(f"Current pick: {draft_state_early.current_pick}")
print(f"Dynamic VBD enabled: {transformer.enabled}")

# Check baseline overrides calculation manually
baseline_overrides = transformer._compute_adjustments(df.copy(), probabilities_balanced)
print(f"\nGenerated baseline overrides: {baseline_overrides}")

# Apply dynamic VBD transformation
df_scenario_a = transformer.transform(df.copy(), probabilities_balanced, draft_state_early)

# Compare a few key values
print(f"\nCOMPARISON CHECK:")
print("Original vs Dynamic VBD values for top 5 RBs:")
original_rbs = df[df['POSITION'] == 'RB'].nlargest(5, 'VBD_BEER')[['PLAYER', 'VBD_BEER']]
dynamic_rbs = df_scenario_a[df_scenario_a['POSITION'] == 'RB'].nlargest(5, 'VBD_BEER')[['PLAYER', 'VBD_BEER']]

for i in range(min(5, len(original_rbs))):
    orig_player = original_rbs.iloc[i]['PLAYER']
    orig_vbd = original_rbs.iloc[i]['VBD_BEER']
    
    dyn_row = dynamic_rbs[dynamic_rbs['PLAYER'] == orig_player]
    if not dyn_row.empty:
        dyn_vbd = dyn_row.iloc[0]['VBD_BEER']
        change = dyn_vbd - orig_vbd
        print(f"  {orig_player}: {orig_vbd:.2f} → {dyn_vbd:.2f} (Δ{change:+.2f})")

# Rank by dynamic VBD (use BEER as the dynamic method)
dynamic_col = 'VBD_BEER'  # Dynamic VBD uses BEER method
df_scenario_a_ranked = df_scenario_a.sort_values(dynamic_col, ascending=False).reset_index(drop=True)
df_scenario_a_ranked['DYNAMIC_RANK_A'] = range(1, len(df_scenario_a_ranked) + 1)

print(f"\n🎯 SCENARIO A: Early Draft - Balanced Expectations (Top 20)")
print("=" * 70)
scenario_a_display = df_scenario_a_ranked[['DYNAMIC_RANK_A', 'PLAYER', 'POSITION', dynamic_col]].head(20)
print(scenario_a_display.to_string(index=False))

🔍 DEBUGGING DYNAMIC VBD SCENARIO A
Probabilities: {'RB': 0.35, 'WR': 0.3, 'QB': 0.15, 'TE': 0.15, 'DST': 0.03, 'K': 0.02}
Horizon picks: 28
Current pick: 15
Dynamic VBD enabled: True

Generated baseline overrides: {'RB': {'BEER': 99.62392499476161}, 'QB': {'BEER': 173.13146026695136}, 'WR': {'BEER': 115.80996912594074}, 'TE': {'BEER': 70.09481390882368}, 'DST': {'BEER': 97.45172176373951}, 'K': {'BEER': np.float64(42.2)}}

COMPARISON CHECK:
Original vs Dynamic VBD values for top 5 RBs:
  Saquon Barkley PHI: 159.37 → 173.55 (Δ+14.18)
  Bijan Robinson ATL: 149.19 → 163.37 (Δ+14.18)
  Jahmyr Gibbs DET: 145.43 → 159.61 (Δ+14.18)
  Derrick Henry BAL: 134.76 → 148.94 (Δ+14.18)
  Christian McCaffrey SF: 114.87 → 129.05 (Δ+14.18)

🎯 SCENARIO A: Early Draft - Balanced Expectations (Top 20)
 DYNAMIC_RANK_A                 PLAYER POSITION  VBD_BEER
              1      Lamar Jackson BAL       QB    194.39
              2         Josh Allen BUF       QB    193.04
              3        Jalen Hurts

### Scenario B: Mid Draft (Pick 75) - RB Run Expected

**Context:** You're at pick 75 (round 6 of 14-team draft). Teams are now filling specific roster needs, and you expect a run on running backs because many teams still need their RB2.

**Draft Probability Assumptions:**
- **RB: 60%** - Heavy expectation of RB picks due to positional scarcity
- **WR: 25%** - Some WR picks but less focus than RBs
- **QB: 5%** - Most teams have their QB by now
- **TE: 8%** - Teams filling TE needs
- **DST/K: 2%** - Still too early for most teams

**Expected Impact:** Dynamic VBD should significantly boost RB values since they're expected to be drafted much more heavily than the static baseline suggests. This represents "positional scarcity" - when a position becomes more valuable due to supply/demand dynamics.

In [16]:
# Scenario B: Mid-draft with expected RB run
probabilities_rb_run = create_probability_forecast(
    horizon_picks=14,  # Looking ahead 1 round
    position_probs={
        'RB': 0.60,  # Heavy RB expectation
        'WR': 0.25,
        'QB': 0.05,
        'TE': 0.08,
        'DST': 0.01,
        'K': 0.01
    }
)

draft_state_mid = create_draft_state(
    current_pick=75,
    drafted_players=[f'Player_{i}' for i in range(74)]  # 74 players already drafted
)

print("🔍 DEBUGGING DYNAMIC VBD SCENARIO B")
print("=" * 50)
print(f"Probabilities: {probabilities_rb_run.position_probs}")
print(f"Horizon picks: {probabilities_rb_run.horizon_picks}")
print(f"Current pick: {draft_state_mid.current_pick}")

# Check baseline overrides calculation manually
baseline_overrides_b = transformer._compute_adjustments(df.copy(), probabilities_rb_run)
print(f"\nGenerated baseline overrides: {baseline_overrides_b}")

# Calculate expected picks for comparison
for pos, prob in probabilities_rb_run.position_probs.items():
    expected_picks = prob * probabilities_rb_run.horizon_picks
    adjustment = transformer.scale * np.tanh(expected_picks / transformer.kappa)
    print(f"  {pos}: prob={prob:.2f}, expected_picks={expected_picks:.1f}, adjustment={adjustment:.3f}")

# Apply dynamic VBD transformation
df_scenario_b = transformer.transform(df.copy(), probabilities_rb_run, draft_state_mid)

# Compare values vs scenario A
print(f"\nCOMPARISON: Scenario A vs B (Top 5 RBs):")
for i in range(min(5, len(original_rbs))):
    orig_player = original_rbs.iloc[i]['PLAYER']
    
    # Get scenario A value
    a_row = df_scenario_a[df_scenario_a['PLAYER'] == orig_player]
    a_vbd = a_row.iloc[0]['VBD_BEER'] if not a_row.empty else 0
    
    # Get scenario B value  
    b_row = df_scenario_b[df_scenario_b['PLAYER'] == orig_player]
    b_vbd = b_row.iloc[0]['VBD_BEER'] if not b_row.empty else 0
    
    diff = b_vbd - a_vbd
    print(f"  {orig_player}: A={a_vbd:.2f}, B={b_vbd:.2f} (Δ{diff:+.2f})")

# Rank by dynamic VBD
df_scenario_b_ranked = df_scenario_b.sort_values(dynamic_col, ascending=False).reset_index(drop=True)
df_scenario_b_ranked['DYNAMIC_RANK_B'] = range(1, len(df_scenario_b_ranked) + 1)

print(f"\n🏃 SCENARIO B: Mid Draft - RB Run Expected (Top 20)")
print("=" * 70)
scenario_b_display = df_scenario_b_ranked[['DYNAMIC_RANK_B', 'PLAYER', 'POSITION', dynamic_col]].head(20)
print(scenario_b_display.to_string(index=False))

🔍 DEBUGGING DYNAMIC VBD SCENARIO B
Probabilities: {'RB': 0.6, 'WR': 0.25, 'QB': 0.05, 'TE': 0.08, 'DST': 0.01, 'K': 0.01}
Horizon picks: 14
Current pick: 75

Generated baseline overrides: {'RB': {'BEER': 100.04029208680365}, 'QB': {'BEER': 238.8741331426154}, 'WR': {'BEER': 118.10942556017984}, 'TE': {'BEER': 74.51689139208662}, 'DST': {'BEER': 98.49005486279509}, 'K': {'BEER': np.float64(42.2)}}
  RB: prob=0.60, expected_picks=8.4, adjustment=23.322
  WR: prob=0.25, expected_picks=3.5, adjustment=15.109
  QB: prob=0.05, expected_picks=0.7, adjustment=3.477
  TE: prob=0.08, expected_picks=1.1, adjustment=5.508
  DST: prob=0.01, expected_picks=0.1, adjustment=0.700
  K: prob=0.01, expected_picks=0.1, adjustment=0.700

COMPARISON: Scenario A vs B (Top 5 RBs):
  Saquon Barkley PHI: A=173.55, B=173.13 (Δ-0.42)
  Bijan Robinson ATL: A=163.37, B=162.95 (Δ-0.42)
  Jahmyr Gibbs DET: A=159.61, B=159.19 (Δ-0.42)
  Derrick Henry BAL: A=148.94, B=148.52 (Δ-0.42)
  Christian McCaffrey SF: A=129.05,

### Scenario C: Late Draft (Pick 150) - WR/TE Focus

**Context:** You're at pick 150 (round 11 of 14-team draft). Most starting lineups are filled, and teams are looking for depth at skill positions, particularly WR and TE where there's more opportunity for breakout players.

**Draft Probability Assumptions:**
- **RB: 15%** - Most teams have enough RB depth by now
- **WR: 45%** - High focus on WR depth and potential breakouts
- **QB: 10%** - Backup QBs and streaming options
- **TE: 25%** - Teams looking for TE depth or upgrades
- **DST/K: 5%** - Some teams start taking DST/K in later rounds

**Expected Impact:** Dynamic VBD should boost WR and TE values since they're expected to be drafted more heavily than normal, while RB values should decrease since they're in less demand at this stage of the draft.

In [17]:
# Scenario C: Late draft with WR/TE focus
probabilities_wr_te = create_probability_forecast(
    horizon_picks=7,  # Looking ahead half round
    position_probs={
        'RB': 0.15,
        'WR': 0.45,  # Heavy WR focus
        'QB': 0.10,
        'TE': 0.25,  # TE run expected
        'DST': 0.03,
        'K': 0.02
    }
)

draft_state_late = create_draft_state(
    current_pick=150,
    drafted_players=[f'Player_{i}' for i in range(149)]  # 149 players already drafted
)

# Apply dynamic VBD transformation
df_scenario_c = transformer.transform(df.copy(), probabilities_wr_te, draft_state_late)

# Rank by dynamic VBD
df_scenario_c_ranked = df_scenario_c.sort_values(dynamic_col, ascending=False).reset_index(drop=True)
df_scenario_c_ranked['DYNAMIC_RANK_C'] = range(1, len(df_scenario_c_ranked) + 1)

print("📡 SCENARIO C: Late Draft - WR/TE Focus (Top 20)")
print("=" * 70)
scenario_c_display = df_scenario_c_ranked[['DYNAMIC_RANK_C', 'PLAYER', 'POSITION', dynamic_col]].head(20)
print(scenario_c_display.to_string(index=False))

📡 SCENARIO C: Late Draft - WR/TE Focus (Top 20)
 DYNAMIC_RANK_C                 PLAYER POSITION  VBD_BEER
              1     Saquon Barkley PHI       RB    162.42
              2     Bijan Robinson ATL       RB    152.24
              3       Jahmyr Gibbs DET       RB    148.48
              4      Derrick Henry BAL       RB    137.81
              5      Lamar Jackson BAL       QB    128.65
              6         Josh Allen BUF       QB    127.30
              7 Christian McCaffrey SF       RB    117.92
              8         Josh Jacobs GB       RB    115.83
              9      De'Von Achane MIA       RB    112.71
             10      Ja'Marr Chase CIN       WR    112.61
             11        Jalen Hurts PHI       QB    110.70
             12       Ashton Jeanty LV       RB    106.88
             13     Jayden Daniels WAS       QB    106.58
             14    Jonathan Taylor IND       RB    105.61
             15     Kyren Williams LAR       RB    102.67
             16         

## 5. Comparative Analysis: Static vs Dynamic Rankings

In [18]:
# Merge all rankings for comparison
comparison_df = df_static_ranked[['PLAYER', 'POSITION', 'STATIC_RANK']].copy()

# Add dynamic rankings
if len(df_scenario_a_ranked) > 0:
    comparison_df = comparison_df.merge(
        df_scenario_a_ranked[['PLAYER', 'DYNAMIC_RANK_A']], on='PLAYER', how='left'
    )
if len(df_scenario_b_ranked) > 0:
    comparison_df = comparison_df.merge(
        df_scenario_b_ranked[['PLAYER', 'DYNAMIC_RANK_B']], on='PLAYER', how='left'
    )
if len(df_scenario_c_ranked) > 0:
    comparison_df = comparison_df.merge(
        df_scenario_c_ranked[['PLAYER', 'DYNAMIC_RANK_C']], on='PLAYER', how='left'
    )

# Calculate rank changes
if 'DYNAMIC_RANK_A' in comparison_df.columns:
    comparison_df['CHANGE_A'] = comparison_df['STATIC_RANK'] - comparison_df['DYNAMIC_RANK_A']
if 'DYNAMIC_RANK_B' in comparison_df.columns:
    comparison_df['CHANGE_B'] = comparison_df['STATIC_RANK'] - comparison_df['DYNAMIC_RANK_B']
if 'DYNAMIC_RANK_C' in comparison_df.columns:
    comparison_df['CHANGE_C'] = comparison_df['STATIC_RANK'] - comparison_df['DYNAMIC_RANK_C']

print("📊 RANKING COMPARISON: Static vs Dynamic (Top 30)")
print("=" * 80)
print("Positive change = player ranked higher in dynamic VBD")
print("Negative change = player ranked lower in dynamic VBD")
print()

# Show top 30 with changes
display_cols = ['PLAYER', 'POSITION', 'STATIC_RANK']
if 'DYNAMIC_RANK_A' in comparison_df.columns:
    display_cols.extend(['DYNAMIC_RANK_A', 'CHANGE_A'])
if 'DYNAMIC_RANK_B' in comparison_df.columns:
    display_cols.extend(['DYNAMIC_RANK_B', 'CHANGE_B'])
if 'DYNAMIC_RANK_C' in comparison_df.columns:
    display_cols.extend(['DYNAMIC_RANK_C', 'CHANGE_C'])

comparison_display = comparison_df[display_cols].head(30)
print(comparison_display.to_string(index=False))

📊 RANKING COMPARISON: Static vs Dynamic (Top 30)
Positive change = player ranked higher in dynamic VBD
Negative change = player ranked lower in dynamic VBD

                PLAYER POSITION  STATIC_RANK  DYNAMIC_RANK_A  CHANGE_A  DYNAMIC_RANK_B  CHANGE_B  DYNAMIC_RANK_C  CHANGE_C
    Saquon Barkley PHI       RB            1               4        -3               1         0               1         0
    Bijan Robinson ATL       RB            2               6        -4               2         0               2         0
      Jahmyr Gibbs DET       RB            3               7        -4               3         0               3         0
     Derrick Henry BAL       RB            4               9        -5               4         0               4         0
     Lamar Jackson BAL       QB            5               1         4               5         0               5         0
        Josh Allen BUF       QB            6               2         4               7        -1         

## 6. Biggest Movers Analysis

In [19]:
# Analyze biggest rank changes (filter out DST/K with missing names)
print("🚀 BIGGEST MOVERS ANALYSIS")
print("=" * 50)
print("Note: Filtering out players with missing names (typically DST/K) for cleaner analysis")
print()

# Filter out rows with NaN player names for cleaner analysis
valid_players = comparison_df.dropna(subset=['PLAYER'])

if 'CHANGE_A' in valid_players.columns and len(valid_players) > 0:
    print("📊 SCENARIO A (Balanced) - Biggest Movers")
    print("-" * 45)
    
    # Biggest risers (positive change = ranked higher in dynamic)
    risers_a = valid_players.nlargest(10, 'CHANGE_A')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_A', 'CHANGE_A']]
    print("Top 10 Risers (ranked higher with Dynamic VBD):")
    print(risers_a.to_string(index=False))
    
    # Biggest fallers (negative change = ranked lower in dynamic)
    fallers_a = valid_players.nsmallest(10, 'CHANGE_A')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_A', 'CHANGE_A']]
    print("\nTop 10 Fallers (ranked lower with Dynamic VBD):")
    print(fallers_a.to_string(index=False))

if 'CHANGE_B' in valid_players.columns and len(valid_players) > 0:
    print("\n\n🏃 SCENARIO B (RB Run) - Biggest Movers")
    print("-" * 45)
    
    # Biggest risers
    risers_b = valid_players.nlargest(10, 'CHANGE_B')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_B', 'CHANGE_B']]
    print("Top 10 Risers (ranked higher with Dynamic VBD):")
    print(risers_b.to_string(index=False))
    
    # Biggest fallers  
    fallers_b = valid_players.nsmallest(10, 'CHANGE_B')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_B', 'CHANGE_B']]
    print("\nTop 10 Fallers (ranked lower with Dynamic VBD):")
    print(fallers_b.to_string(index=False))

if 'CHANGE_C' in valid_players.columns and len(valid_players) > 0:
    print("\n\n📡 SCENARIO C (WR/TE Focus) - Biggest Movers")
    print("-" * 45)
    
    # Biggest risers
    risers_c = valid_players.nlargest(10, 'CHANGE_C')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_C', 'CHANGE_C']]
    print("Top 10 Risers (ranked higher with Dynamic VBD):")
    print(risers_c.to_string(index=False))
    
    # Biggest fallers
    fallers_c = valid_players.nsmallest(10, 'CHANGE_C')[['PLAYER', 'POSITION', 'STATIC_RANK', 'DYNAMIC_RANK_C', 'CHANGE_C']]
    print("\nTop 10 Fallers (ranked lower with Dynamic VBD):")
    print(fallers_c.to_string(index=False))

🚀 BIGGEST MOVERS ANALYSIS
Note: Filtering out players with missing names (typically DST/K) for cleaner analysis

📊 SCENARIO A (Balanced) - Biggest Movers
---------------------------------------------
Top 10 Risers (ranked higher with Dynamic VBD):
                    PLAYER POSITION  STATIC_RANK  DYNAMIC_RANK_A  CHANGE_A
Anthony Richardson Sr. IND       QB          258              98       160
        Russell Wilson NYG       QB          268             115       153
         Aaron Rodgers PIT       QB          206              66       140
          Cameron Ward TEN       QB          185              53       132
           Sam Darnold SEA       QB          179              51       128
           Tyler Shough NO       QB          292             175       117
             Geno Smith LV       QB          161              48       113
           Bryce Young CAR       QB          153              45       108
     Michael Penix Jr. ATL       QB          154              46       108
  

## 7. Position-Specific Impact Analysis

In [20]:
# Analyze impact by position
if 'CHANGE_A' in comparison_df.columns:
    print("📈 POSITION-SPECIFIC IMPACT ANALYSIS")
    print("=" * 60)
    
    # Calculate average change by position for each scenario
    # Fix MultiIndex issue by creating separate DataFrames for each scenario
    position_impact_data = []
    
    # Scenario A analysis
    if 'CHANGE_A' in comparison_df.columns:
        pos_stats_a = comparison_df.groupby('POSITION')['CHANGE_A'].agg(['mean', 'std', 'count']).round(2)
        pos_stats_a.columns = ['Mean_A', 'Std_A', 'Count_A']
        position_impact_data.append(pos_stats_a)
    
    # Scenario B analysis
    if 'CHANGE_B' in comparison_df.columns:
        pos_stats_b = comparison_df.groupby('POSITION')['CHANGE_B'].agg(['mean', 'std']).round(2)
        pos_stats_b.columns = ['Mean_B', 'Std_B']
        position_impact_data.append(pos_stats_b)
    
    # Scenario C analysis
    if 'CHANGE_C' in comparison_df.columns:
        pos_stats_c = comparison_df.groupby('POSITION')['CHANGE_C'].agg(['mean', 'std']).round(2)
        pos_stats_c.columns = ['Mean_C', 'Std_C']
        position_impact_data.append(pos_stats_c)
    
    # Combine all scenario data
    if position_impact_data:
        position_impact = position_impact_data[0]
        for additional_data in position_impact_data[1:]:
            position_impact = position_impact.join(additional_data, how='outer')
        
        print("Average rank change by position:")
        print("(Positive = higher in dynamic, Negative = lower in dynamic)")
        print()
        print(position_impact)
        
        # Summary insights
        print("\n💡 KEY INSIGHTS:")
        print("=" * 40)
        
        if 'CHANGE_A' in comparison_df.columns:
            avg_change_by_pos_a = comparison_df.groupby('POSITION')['CHANGE_A'].mean()
            most_helped_a = avg_change_by_pos_a.idxmax()
            most_hurt_a = avg_change_by_pos_a.idxmin()
            print(f"Scenario A (Balanced): {most_helped_a} benefits most (+{avg_change_by_pos_a[most_helped_a]:.1f}), {most_hurt_a} hurt most ({avg_change_by_pos_a[most_hurt_a]:.1f})")
        
        if 'CHANGE_B' in comparison_df.columns:
            avg_change_by_pos_b = comparison_df.groupby('POSITION')['CHANGE_B'].mean()
            most_helped_b = avg_change_by_pos_b.idxmax()
            most_hurt_b = avg_change_by_pos_b.idxmin()
            print(f"Scenario B (RB Run): {most_helped_b} benefits most (+{avg_change_by_pos_b[most_helped_b]:.1f}), {most_hurt_b} hurt most ({avg_change_by_pos_b[most_hurt_b]:.1f})")
        
        if 'CHANGE_C' in comparison_df.columns:
            avg_change_by_pos_c = comparison_df.groupby('POSITION')['CHANGE_C'].mean()
            most_helped_c = avg_change_by_pos_c.idxmax()
            most_hurt_c = avg_change_by_pos_c.idxmin()
            print(f"Scenario C (WR/TE): {most_helped_c} benefits most (+{avg_change_by_pos_c[most_helped_c]:.1f}), {most_hurt_c} hurt most ({avg_change_by_pos_c[most_hurt_c]:.1f})")
    else:
        print("No position impact data available for analysis.")

📈 POSITION-SPECIFIC IMPACT ANALYSIS
Average rank change by position:
(Positive = higher in dynamic, Negative = lower in dynamic)

          Mean_A  Std_A  Count_A  Mean_B  Std_B  Mean_C  Std_C
POSITION                                                      
DST       -28.09  43.66  2370816  -18.66  44.63  -13.97  44.37
K         -31.42  31.85   740880  -21.99  33.17  -17.30  32.82
QB         55.51  49.84       37   11.97  15.65   14.73  16.33
RB          6.06  17.40       72   15.56  16.41    2.33   5.45
TE        -11.70   6.53       40  -12.50   3.34   -4.75   2.83
WR         -7.42   7.65      109   -2.26   4.77    0.89   4.07

💡 KEY INSIGHTS:
Scenario A (Balanced): QB benefits most (+55.5), K hurt most (-31.4)
Scenario B (RB Run): RB benefits most (+15.6), K hurt most (-22.0)
Scenario C (WR/TE): QB benefits most (+14.7), K hurt most (-17.3)


## 8. Export Dynamic VBD Rankings

In [21]:
# Export the comparison and scenario rankings
today = datetime.now().strftime('%Y%m%d_%H%M%S')

# Create output directory if it doesn't exist
output_dir = '../data/output'
os.makedirs(output_dir, exist_ok=True)

# Export comparison table
comparison_file = f'{output_dir}/dynamic_vbd_comparison_{today}.csv'
comparison_df.to_csv(comparison_file, index=False)
print(f"✅ Exported comparison rankings: {comparison_file}")

# Export individual scenario rankings
if len(df_scenario_a_ranked) > 0:
    scenario_a_file = f'{output_dir}/dynamic_vbd_scenario_a_balanced_{today}.csv'
    df_scenario_a_ranked.head(300).to_csv(scenario_a_file, index=False)
    print(f"✅ Exported Scenario A rankings: {scenario_a_file}")

if len(df_scenario_b_ranked) > 0:
    scenario_b_file = f'{output_dir}/dynamic_vbd_scenario_b_rb_run_{today}.csv'
    df_scenario_b_ranked.head(300).to_csv(scenario_b_file, index=False)
    print(f"✅ Exported Scenario B rankings: {scenario_b_file}")

if len(df_scenario_c_ranked) > 0:
    scenario_c_file = f'{output_dir}/dynamic_vbd_scenario_c_wr_te_{today}.csv'
    df_scenario_c_ranked.head(300).to_csv(scenario_c_file, index=False)
    print(f"✅ Exported Scenario C rankings: {scenario_c_file}")

print(f"\n📊 All dynamic VBD test results exported with timestamp: {today}")

✅ Exported comparison rankings: ../data/output/dynamic_vbd_comparison_20250816_132753.csv
✅ Exported Scenario A rankings: ../data/output/dynamic_vbd_scenario_a_balanced_20250816_132753.csv
✅ Exported Scenario B rankings: ../data/output/dynamic_vbd_scenario_b_rb_run_20250816_132753.csv
✅ Exported Scenario C rankings: ../data/output/dynamic_vbd_scenario_c_wr_te_20250816_132753.csv

📊 All dynamic VBD test results exported with timestamp: 20250816_132753


## 9. Cache Performance Analysis

In [22]:
# Check cache performance
final_cache_stats = transformer.get_cache_stats()
print("⚡ DYNAMIC VBD CACHE PERFORMANCE")
print("=" * 50)
print(f"Final cache size: {final_cache_stats['cache_size']} entries")
print(f"Cache keys: {final_cache_stats['cache_keys']}")

# Clear cache to demonstrate functionality
transformer.clear_cache()
cleared_stats = transformer.get_cache_stats()
print(f"\nAfter clearing - cache size: {cleared_stats['cache_size']}")

print("\n✅ Dynamic VBD testing completed successfully!")
print("\n💡 SUMMARY:")
print("- Static VBD provides baseline rankings")
print("- Dynamic VBD adjusts based on draft probability forecasts")
print("- Different scenarios produce different ranking adjustments")
print("- Position scarcity detection drives value changes")
print("- Cache system optimizes performance for repeated calculations")

⚡ DYNAMIC VBD CACHE PERFORMANCE
Final cache size: 3 entries
Cache keys: ['15_2', '75_74', '150_149']

After clearing - cache size: 0

✅ Dynamic VBD testing completed successfully!

💡 SUMMARY:
- Static VBD provides baseline rankings
- Dynamic VBD adjusts based on draft probability forecasts
- Different scenarios produce different ranking adjustments
- Position scarcity detection drives value changes
- Cache system optimizes performance for repeated calculations
